In [1]:
!pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 3.0 MB/s eta 0:00:00


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scrapy
from scrapy.crawler import CrawlerProcess
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
class ShakespeareSpider(scrapy.Spider):
    """
    Spider for the website: https://allpoetry.com
    Scraping only the most famous poems
    """
    def __init__(self):
        #super(ShakespeareSpider, self).__init__()
        self.name = "Poems"
        self.start_urls = ['https://allpoetry.com']
        self.base_url = 'https://allpoetry.com'
        self.poems = []
        self.counter = 0
    def start_requests(self):
        page1 = 'https://allpoetry.com/William-Shakespeare'
        yield scrapy.Request(page1,callback=self.parse_page, headers={'User-Agent': 
                            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:54.0) Gecko/20100101 Firefox/54.0'
                            })
        
    def parse_page(self,response):
        """
        Processing each page of the website
        """
        
        poems_list_page = response.css('div.author_link_list')
        for items in poems_list_page:

            links = items.css('a')
            for link in links:
                if self.counter >= 0 and self.counter < 49:
                    poem = dict()
                    title = link.xpath('text()').get()
                    href = link.xpath('@href').get()
                    poem['title'] = title
                    print(title)
                    #
                    if href:
                        if "Full title" not in title:
                            yield scrapy.Request(self.base_url+href, callback=self.parse_poem,cb_kwargs={'poem': poem})
        
            
    
        
    
    def parse_poem(self,response,poem):
        """
        getting the body of the poem
        """
        if len(self.poems) > 47:
            #assert len(self.poems) > 0
            df = pd.DataFrame.from_dict(self.poems)
            print("printing DataFrame")
            print(df.head())
            df.to_csv('Shakespeare_poems.csv')
        else:
            poem_body = response.css('div.poem_body')
            text = poem_body.xpath('./div[2]//p//text()').extract()
            if len(text) == 0:
                text = poem_body.xpath('./div[2]//text()').extract()
            poem['poem_body'] = text
            print(poem.keys())
            self.poems.append(poem)
            self.counter += 1
            print(len(self.poems))

    #print(self.poems)
        

In [4]:
process = CrawlerProcess(settings={
    'DOWNLOAD_DELAY': 5,
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:54.0) Gecko/20100101 Firefox/54.0'
})
process.crawl(ShakespeareSpider)
process.start()

2023-08-07 23:23:31 [scrapy.utils.log] INFO: Scrapy 2.10.0 started (bot: scrapybot)
2023-08-07 23:23:31 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.1 30 May 2023), cryptography 41.0.1, Platform Linux-5.15.120+-x86_64-with-glibc2.31
2023-08-07 23:23:31 [scrapy.addons] INFO: Enabled addons:
[]
2023-08-07 23:23:31 [scrapy.crawler] INFO: Overridden settings:
{'DOWNLOAD_DELAY': 5,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:54.0) '
               'Gecko/20100101 Firefox/54.0'}
2023-08-07 23:23:31 [py.warnings] WARNING: /opt/conda/lib/python3.10/site-packages/scrapy/utils/request.py:248: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if yo

A Lover's Complaint
A Madrigal
All The World's A Stage
Aubade
Blow, Blow, Thou Winter Wind
Bridal Song
Carpe Diem
Dear Friend
Dirge
Dirge Of The Three Queens
Fairy Land I
Fairy Land II
Fairy Land III
Fairy Land IV
Fairy Land V
Fear No More
Fidele
Friends, Romans, countrymen, lend me your ears
From The Rape Of Lucrece
From Venus And Adonis
Helen's Soliloqy (All's Well That Ends Well)
How Like A Winter Hath My Absence Been
It Was A Lover And His Lass
Juliet's Soliloquy
Love
Macbeth - Three witches casting a spell
Now The Hungry Lion Roars
Now, my co-mates and brothers in exile
O Never Say That I Was False of Heart
Orpheus With His Lute Made Trees
Sigh No More
Silvia
Sir, in my heart there was a kind of fighting (Hamlet Act V, Scene II)
Some Say That Ever ‘Gainst That Season Comes (Hamlet, Act I, Scene I)
Song of the Witches: Double, double toil and trouble
Sonnet 100: "Where art thou Muse that thou forget'st so long,..."
Sonnet 101: "O truant Muse what shall be thy amends..."
Sonnet 102:

2023-08-07 23:23:37 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://allpoetry.com/poem/8449711-Dirge-by-William-Shakespeare> (referer: https://allpoetry.com/William-Shakespeare)


dict_keys(['title', 'poem_body'])
1


2023-08-07 23:23:43 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://allpoetry.com/poem/16377694-Dear-Friend-by-William-Shakespeare> (referer: https://allpoetry.com/William-Shakespeare)


...

dict_keys(['title', 'poem_body'])
47


2023-08-07 23:28:19 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://allpoetry.com/Fairy-Land-I> (referer: https://allpoetry.com/William-Shakespeare)


dict_keys(['title', 'poem_body'])
48


2023-08-07 23:28:26 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://allpoetry.com/Dirge-Of-The-Three-Queens> (referer: https://allpoetry.com/William-Shakespeare)
2023-08-07 23:28:26 [scrapy.core.engine] INFO: Closing spider (finished)
2023-08-07 23:28:26 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 44073,
 'downloader/request_count': 50,
 'downloader/request_method_count/GET': 50,
 'downloader/response_bytes': 634782,
 'downloader/response_count': 50,
 'downloader/response_status_count/200': 50,
 'elapsed_time_seconds': 294.97617,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2023, 8, 7, 23, 28, 26, 219306),
 'httpcompression/response_bytes': 2002775,
 'httpcompression/response_count': 50,
 'log_count/DEBUG': 53,
 'log_count/INFO': 14,
 'log_count/WARNING': 1,
 'memusage/max': 278990848,
 'memusage/startup': 264671232,
 'request_depth_max': 1,
 'response_received_count': 50,
 'scheduler/dequeued': 50,
 'scheduler/dequeued/me

printing DataFrame
                          title  \
0                         Dirge   
1                   Dear Friend   
2                    Carpe Diem   
3                   Bridal Song   
4  Blow, Blow, Thou Winter Wind   

                                           poem_body  
0  [COME away, come away, death, , \n   And in sa...  
1  [When to the session of sweet silent thought, ...  
2  [O mistress mine, where are you roaming? , \nO...  
3  [ROSES, their sharp spines being gone, , \nNot...  
4  [Blow, blow, thou winter wind , \nThou art not...  
